In [2]:
import os
import random
from datetime import datetime
import torch
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter
from data.datasets import AMIGOS, series_collate
from architecture.MainNetwork import MainNetwork
from utils import *

In [3]:
torch.backends.cudnn.enabled = False
loader_kwargs = {'num_workers': 4, 'pin_memory': True, 'shuffle': True, 'drop_last': True}
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'
torch.cuda.set_device(1)
torch.cuda.current_device()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
savemodel = 'models/'
if not os.path.exists(savemodel):
    os.makedirs(savemodel)
root_path = 'data/face_segments'
labels_path = 'data/Data_Preprocessed_segmented.json'
vids_dir = 'data/vids_segments'
remove_mov = 'data/ignore_mov.json'
num_class = 4096
batch_size = 4
learning_rate = 1e-05
epochs = 20
alpha = 2
beta = 1
gamma = 1
scale_factor = 0.00018033097575909232
downsample = 6



In [4]:
log_dir = 'log'
log_writer = SummaryWriter(os.path.join(*[log_dir, 'AMIGOS', 'Train', datetime.now().strftime('%b%d_%H-%M-%S')]))

x_transform = transforms.Compose([
    transforms.ColorJitter(0.2, 0.2, 0.2),
    transforms.RandomHorizontalFlip(),
    transforms.Normalize([0.4168, 0.3074, 0.2607], [0.2426, 0.1997, 0.1870])
])

y_transform = transforms.Compose([
    torch.FloatTensor
])

train_dataset = AMIGOS(
    root_path=root_path,
    labels_path=labels_path,
    vids_dir=vids_dir,
    remove_mov=remove_mov,
    x_transform=x_transform,
    y_transform=y_transform,
    downsample=downsample
)

x_transform = transforms.Compose([
    transforms.Normalize([0.4168, 0.3074, 0.2607], [0.2426, 0.1997, 0.1870])
])

val_dataset = AMIGOS(
    root_path=root_path,
    labels_path=labels_path,
    vids_dir=vids_dir,
    remove_mov=remove_mov,
    x_transform=x_transform,
    y_transform=y_transform,
    downsample=downsample
)

In [4]:
output_names = ['AR', 'ECG']
for uid in train_dataset.data.keys():
    train_idx = [idx[0] for idx in train_dataset.idxs if idx[1] != uid]
    val_idx = [idx[0] for idx in train_dataset.idxs if idx[1] == uid]

    actual_train = random.sample(train_idx, len(train_idx) // 5)
    print('Training UID {} with {} samples'.format(uid, len(actual_train)))

    train_set = data.Subset(train_dataset, actual_train)
    val_set = data.Subset(val_dataset, val_idx)
    train_loader = torch.utils.data.DataLoader(
        train_set,
        batch_size=batch_size,
        collate_fn=series_collate,
        **loader_kwargs
    )
    test_loader = torch.utils.data.DataLoader(
        val_set,
        batch_size=batch_size,
        collate_fn=series_collate,
        **loader_kwargs
    )


    # training
    lr = learning_rate
    model = MainNetwork(num_class).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=5e-3)
    scaler = torch.cuda.amp.GradScaler()
    loss_hist = []
    
    iter_idx = 0
    for epoch in range(epochs):
        model.train()
        if (epoch % 5 == 0) and (epoch != 0):
            lr *= 0.1
            optimizer.param_groups[0]['lr'] = lr
        num_iter = len(train_loader)
        for batch_idx, (inputs, labels1, labels2, _) in enumerate(train_loader):
            iter_idx = (epoch * num_iter) + batch_idx
            inputs, labels1, labels2 = inputs.cuda(), labels1.cuda(), labels2.cuda()
            losses = []
            optimizer.zero_grad()
            with torch.autocast(device.type):
                outputs = model(inputs)
                labels = [labels1, labels2]
                for i in range(len(labels)):
                    if output_names[i] == 'ECG':
                        mae, mse, rmse, pcc, ccc = eval_metrics(outputs[i].permute(0, 2, 1).reshape((batch_size * 2560, 2)) * scale_factor, labels[i].permute(0, 2, 1).reshape((batch_size * 2560, 2)) * scale_factor)
                    else:
                        mae, mse, rmse, pcc, ccc = eval_metrics(outputs[i], labels[i])
                    loss = (1-ccc).mean() + alpha * rmse
                    losses.append(loss)

                    logging('Train', output_names[i], log_writer, loss, mae, mse, rmse, pcc, ccc, iter_idx)
                    print('\n Epoch [{}/{}] Iter[{}/{}] Mode: {} \t loss: {:.2f} \t MAE: {:.2f} \t MSE: {:.2f} \t RMSE: {:.2f} \t PCC:{} \t CCC:{} '.format(
                        epoch,
                        epochs,
                        batch_idx + 1,
                        num_iter,
                        output_names[i],
                        loss.item(),
                        mae.item(),
                        mse.item(),
                        rmse.item(),
                        ['%.2f' % elem for elem in pcc.tolist()],
                        ['%.2f' % elem for elem in ccc.tolist()],
                    ))

            loss = beta * losses[0] + gamma * losses[1]
            log_writer.add_scalar('TotalLoss/{}'.format('Train'), loss, iter_idx)
            print('\n Epoch [{}/{}] Iter[{}/{}]\t TotalLoss: {:.2f}'.format(
                epoch,
                epochs,
                batch_idx + 1,
                num_iter,
                loss.item()
            ))
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            # release GPU memory
            del inputs, labels, labels1, labels2, outputs
            torch.cuda.empty_cache()
        
        # validation
        y_pred_AR, y_true_AR, y_pred_ECG, y_true_ECG = run_val(model, test_loader, batch_size)
        losses = val_log(log_writer, alpha, scale_factor, y_pred_AR, y_true_AR, y_pred_ECG, y_true_ECG, epoch)
        loss = beta * losses[0] + gamma * losses[1]
        log_writer.add_scalar('TotalLoss/{}'.format('Validation'), loss, epoch)
        print('\n Epoch [{}/{}] \t TotalLoss: {:.2f}'.format(
            epoch,
            epochs,
            loss.item()
        ))
        log_writer.flush()
        print('\n')
        del losses, y_pred_AR, y_true_AR, y_pred_ECG, y_true_ECG
        torch.cuda.empty_cache()
        loss_hist.append(loss)
        if loss_hist[-1] == min(loss_hist):
            torch.save({
                'epoch': epoch,
                'model': model.state_dict(),
            },
                os.path.join(savemodel, 'pid_{}.pth.tar'.format(uid))
            )
        log_writer.flush()
        
del model
log_writer.close()

Training UID 37 with 1098 samples

 Epoch [0/20] Iter[1/274] Mode: AR 	 loss: 1.27 	 MAE: 0.12 	 MSE: 0.02 	 RMSE: 0.14 	 PCC:['0.18', '-0.23'] 	 CCC:['0.01', '-0.00'] 

 Epoch [0/20] Iter[1/274] Mode: ECG 	 loss: 1.02 	 MAE: 0.01 	 MSE: 0.00 	 RMSE: 0.01 	 PCC:['0.00', '0.01'] 	 CCC:['0.00', '0.00'] 

 Epoch [0/20] Iter[1/274]	 TotalLoss: 2.30

 Epoch [0/20] Iter[2/274] Mode: AR 	 loss: 1.34 	 MAE: 0.13 	 MSE: 0.03 	 RMSE: 0.16 	 PCC:['-0.24', '0.06'] 	 CCC:['-0.04', '0.01'] 

 Epoch [0/20] Iter[2/274] Mode: ECG 	 loss: 1.03 	 MAE: 0.01 	 MSE: 0.00 	 RMSE: 0.02 	 PCC:['0.01', '-0.01'] 	 CCC:['0.00', '-0.00'] 

 Epoch [0/20] Iter[2/274]	 TotalLoss: 2.37

 Epoch [0/20] Iter[3/274] Mode: AR 	 loss: 1.32 	 MAE: 0.14 	 MSE: 0.02 	 RMSE: 0.15 	 PCC:['-0.16', '-0.34'] 	 CCC:['-0.02', '-0.01'] 

 Epoch [0/20] Iter[3/274] Mode: ECG 	 loss: 1.03 	 MAE: 0.01 	 MSE: 0.00 	 RMSE: 0.01 	 PCC:['0.00', '0.01'] 	 CCC:['0.00', '0.00'] 

 Epoch [0/20] Iter[3/274]	 TotalLoss: 2.35

 Epoch [0/20] Iter[4/2

In [34]:
log_dir = 'log'
log_writer = SummaryWriter(os.path.join(*[log_dir, 'AMIGOS', 'Test', datetime.now().strftime('%b%d_%H-%M-%S_eval')]))

model_lst = os.listdir(savemodel)
model_lst = [os.path.join(savemodel, fname) for fname in model_lst]

y_pred_ARs = []
y_true_ARs = []
y_pred_ECGs = []
y_true_ECGs = []
for model_path in model_lst:
    test_loader = eval_dataloader(model_path, val_dataset, batch_size, loader_kwargs)
    
    print('AMIGO {}: Test samples: {}'.format(uid, len(test_loader)))
    if len(test_loader) == 0:
        continue
    state_dicts = torch.load(model_path)
    model.load_state_dict(state_dicts['model'])

    y_pred_AR, y_true_AR, y_pred_ECG, y_true_ECG = run_val(model, test_loader, batch_size)
    y_pred_ARs.append(y_pred_AR.cpu())
    y_true_ARs.append(y_true_AR.cpu())
    y_pred_ECGs.append(y_pred_ECG.cpu())
    y_true_ECGs.append(y_true_ECG.cpu())
    torch.cuda.empty_cache()

del model

y_pred_ARs = torch.cat(y_pred_ARs)
y_true_ARs = torch.cat(y_true_ARs)
y_pred_ECGs = torch.cat(y_pred_ECGs)
y_true_ECGs = torch.cat(y_true_ECGs)
val_log(log_writer, alpha, scale_factor, y_pred_ARs, y_true_ARs, y_pred_ECGs, y_true_ECGs)

log_writer.flush()
log_writer.close()

AMIGO 37: Test samples: 68


In [26]:
import numpy as np
AR = [x[0] for x in val_dataset.labels]
ECG = [x[1] for x in val_dataset.labels]
AR = np.array(AR).ravel()
ECG = np.array(ECG).ravel()
print('Scale: ', ((AR.max() - AR.min()) / (ECG.max() - ECG.min())))

Scale:  0.00018033097575909232


In [14]:
import os
os.path.exists('requirements.txt')

True